In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [54]:
test_data = pd.read_csv('C:/Users/atalb/Documents/Coding/MLandAI/MLPracticeExercises/Datasets/StickerSales/test.csv')
train_data = pd.read_csv('C:/Users/atalb/Documents/Coding/MLandAI/MLPracticeExercises/Datasets/StickerSales/train.csv')

test_data.head()
# train_data.head()


,id,date,country,store,product
0,230130,2017-01-01,Canada,Discount Stickers,Holographic Goose
1,230131,2017-01-01,Canada,Discount Stickers,Kaggle
2,230132,2017-01-01,Canada,Discount Stickers,Kaggle Tiers
3,230133,2017-01-01,Canada,Discount Stickers,Kerneler
4,230134,2017-01-01,Canada,Discount Stickers,Kerneler Dark Mode


In [55]:
train_data.describe()

,id,num_sold
count,230130.000000,221259.000000
mean,115064.500000,752.527382
std,66432.953062,690.165445
min,0.000000,5.000000
25%,57532.250000,219.000000
50%,115064.500000,605.000000
75%,172596.750000,1114.000000
max,230129.000000,5939.000000


In [56]:

train_data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        230130 non-null  int64  
 1   date      230130 non-null  object 
 2   country   230130 non-null  object 
 3   store     230130 non-null  object 
 4   product   230130 non-null  object 
 5   num_sold  221259 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.5+ MB


In [63]:
train_data.isnull().sum()


id                    0
num_sold           8871
country_encoded       0
store_encoded         0
product_encoded       0
year                  0
month                 0
day                   0
day_of_week           0
dtype: int64

In [65]:
train_data.dropna(inplace=True)
train_data.isnull().sum()



id                 0
num_sold           0
country_encoded    0
store_encoded      0
product_encoded    0
year               0
month              0
day                0
day_of_week        0
dtype: int64

In [66]:
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Create label encoders for each categorical column
le_country = LabelEncoder()
le_store = LabelEncoder()
le_product = LabelEncoder()

# Fit and transform categorical columns in train data
train_data['country_encoded'] = le_country.fit_transform(train_data['country'])
train_data['store_encoded'] = le_store.fit_transform(train_data['store'])
train_data['product_encoded'] = le_product.fit_transform(train_data['product'])

# Transform categorical columns in test data using the same encoders
test_data['country_encoded'] = le_country.transform(test_data['country'])
test_data['store_encoded'] = le_store.transform(test_data['store'])
test_data['product_encoded'] = le_product.transform(test_data['product'])

# Convert date column to datetime type and extract useful features
train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

# Extract date features
for df in [train_data, test_data]:
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.dayofweek

# Display the first few rows of encoded train data
train_data.head()

KeyError: 'country'

In [67]:
test_data.head()

,id,country_encoded,store_encoded,product_encoded,year,month,day,day_of_week
0,230130,0,0,0,2017,1,1,6
1,230131,0,0,1,2017,1,1,6
2,230132,0,0,2,2017,1,1,6
3,230133,0,0,3,2017,1,1,6
4,230134,0,0,4,2017,1,1,6


In [68]:
# Drop original categorical columns from both datasets
columns_to_drop = ['country', 'store', 'product', 'date']

train_data = train_data.drop(columns=columns_to_drop)
test_data = test_data.drop(columns=columns_to_drop)

# Show the first few rows of the cleaned train data
train_data.head()

KeyError: "['country', 'store', 'product', 'date'] not found in axis"

In [69]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Separate features and target
X = train_data.drop(['id', 'num_sold'], axis=1)
y = train_data['num_sold']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the model
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)

# Make predictions on validation set
val_predictions = dt_model.predict(X_val)

# Calculate performance metrics
mse = mean_squared_error(y_val, val_predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, val_predictions)

print(f'Validation RMSE: {rmse:.2f}')
print(f'Validation R2 Score: {r2:.2f}')

# Make predictions on test data
X_test = test_data.drop(['id'], axis=1)
test_predictions = dt_model.predict(X_test)

# Add predictions to test data
test_data['predicted_num_sold'] = test_predictions

# Display first few predictions
test_data[['id', 'predicted_num_sold']].head()

Validation RMSE: 92.21
Validation R2 Score: 0.98


,id,predicted_num_sold
0,230130,139.0
1,230131,738.0
2,230132,597.0
3,230133,366.0
4,230134,392.0


In [41]:
test_pred = test_data[['id', 'predicted_num_sold']]

In [70]:
test_pred.to_csv('C:/Users/atalb/Documents/Coding/MLandAI/MLPracticeExercises/Datasets/StickerSales/test_pred.csv', index=False)